In [3]:
# -*- coding: utf-8 -*- 
import os 
import argparse
import re 
import time 

from modules import folderMoniter, patternRecognizer, metaExtracter
from modules import urlDownload, dropboxInteractor, note_modified

In [4]:
# init 
pattern_recog = patternRecognizer(r'- \[.*\]')  # 检测 - [DOI], 或者- [arxivId]
meta_extracter = metaExtracter()
url_download = urlDownload()

In [17]:
md_file = r"C:\Users\89791\Desktop\Notes&Papers\多模态预训练论文笔记.md"
save_folder = r"C:\Users\89791\Desktop\Notes&Papers\Papers\计算机视觉&自然语言处理论文\多模态"

In [18]:
pdf_link_prefix = "./Papers/计算机视觉&自然语言处理论文/多模态/"

In [19]:
with open(md_file, 'r', encoding='utf-8') as f:
    content = f.read()

In [25]:
m = pattern_recog.findall(content)
print(m)

if m:
    replace_dict = dict()
    for literature in m:
        literature_id = literature.split('[')[-1].split(']')[0]
    
        try:
            bib_dict = meta_extracter.id2bib(literature_id)
            print(bib_dict)
            
            if "pdf_link" in bib_dict.keys():
                pdf_dict = url_download.fetch(bib_dict["pdf_link"])
                if not pdf_dict:
                    pdf_dict = url_download.fetch(literature_id)
            else:
                pdf_dict = url_download.fetch(literature_id)
                
            print("文件的网址是")
            print(pdf_dict['url'])
            
            # Upload attachment and generate shared link
            if "\n" in bib_dict["title"]:
                bib_dict["title"] = re.sub(r' *\n *', ' ', bib_dict["title"])
                
            if ":" in bib_dict["title"]:
                filename = '['+bib_dict['year'][2:]+'] '+bib_dict["title"].replace(':',"")+'.pdf'
            else:
                filename = '['+bib_dict['year'][2:]+'] '+bib_dict["title"]+'.pdf'
            
            save_name = os.path.join(save_folder,filename)
            with open(save_name, "wb") as code:
                 code.write(pdf_dict['pdf'])
                    
            pdf_shared_link = pdf_link_prefix+filename
                
            if 'cited_count' in bib_dict.keys():
                replaced_literature = "- **{}**. {} et.al. **{}**, **{}**, ([pdf]({}))([link]({})), (Citations **{}**).".format(
                    bib_dict['title'], bib_dict["author"].split(" and ")[0], bib_dict['journal'], 
                    bib_dict['year'], pdf_shared_link, bib_dict['url'], bib_dict["cited_count"]
                    )
            else:
                replaced_literature = "- **{}**. {} et.al. **{}**, **{}**, ([pdf]({}))([arxiv]({})), (Citations ).".format(
                    bib_dict['title'], bib_dict["author"].split(" and ")[0], bib_dict['journal'], 
                    bib_dict['year'], pdf_shared_link, bib_dict['url']
                    )
                
            print(replaced_literature)
            replace_dict[literature] = replaced_literature
        except:
            print("")
        
            # replace_dict[literature] = literature
            
    # Modified note
    note_modified(pattern_recog, md_file, **replace_dict)

['- [2111.02358]']
{'journal': 'arxiv', 'url': 'http://arxiv.org/abs/2111.02358v1', 'title': 'VLMo: Unified Vision-Language Pre-Training with\n  Mixture-of-Modality-Experts', 'year': '2021.11', 'author': 'Wenhui Wang and Hangbo Bao and Li Dong and Furu Wei', 'ENTRYTYPE': 'article'}
文件的网址是
https://arxiv.org/pdf/2111.02358.pdf
[21.11]VLMo Unified Vision-Language Pre-Training with Mixture-of-Modality-Experts.pdf
- **VLMo: Unified Vision-Language Pre-Training with Mixture-of-Modality-Experts**. Wenhui Wang et.al. **arxiv**, **2021.11**, ([pdf](./Papers/计算机视觉&自然语言处理论文/多模态/VLMo: Unified Vision-Language Pre-Training with Mixture-of-Modality-Experts.pdf))([arxiv](http://arxiv.org/abs/2111.02358v1)), (Citations ).
